In [1]:
import numpy as np
import csv
import time

In [2]:
# Hyperparameter 설정
RND_MEAN = 0 
RND_STD = 0.0030 # 가중치 파라미터를 초기화 할 때 이용함

LEARNING_RATE = 0.001

In [3]:
def randomize() -> None:
    np.random.seed(time.time())

In [4]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []
    with open('/Users/hyeonjin/workspace/dataset/predicting_pulsar_star/pulsar_stars.csv') as f:
        reader = csv.reader(f)
        next(reader, None)
        for row in reader:
            if '' in row:
                continue
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)

    # with open('/Users/hyeonjin/workspace/dataset/predicting_pulsar_star/pulsar_data_test.csv') as f:
    #     reader = csv.reader(f)
    #     next(reader, None)
    #     for row in reader:
    #         if '' in row:
    #             continue
    #         if row[8] == '1': pulsars.append(row)
    #         else: stars.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1

    pulsar_cnt, star_cnt = len(pulsars), len(stars)
    # print(pulsar_cnt, star_cnt)
    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype=np.float32)
        for n in range(star_cnt):
            data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype=np.float32)
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype=np.float32)
        data[star_cnt:, :] = np.asarray(pulsars, dtype=np.float32)

In [5]:
load_pulsar_dataset(True)

In [6]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [7]:
def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q

def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+fn)
    f1 = 2 * safe_div(recall*precision, recall+precision)
    
    return [accuracy, precision, recall, f1]

In [8]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [9]:
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)    
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

def backprop_postproc_oneline(G_loss, diff):
    return 2 * diff / np.prod(diff.shape)

In [10]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [11]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [12]:
def train_and_test(epoch_count:int, mb_size:int, report:int) -> None:
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses = []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
        
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print(f"Epoch {epoch+1} loss={np.mean(losses):5.3f} result={acc_str}.")
    
    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    print(f"\nFinal Test: final accuracy = {acc_str}")

In [13]:
def init_model() -> None:
    global weight, bias, input_cnt, output_cnt # 전역 변수로 wieght, bias 생성, load_abalone_dataset에서 생성한 input_cnt, output_cnt 사용
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt]) # RND_MEAN, RND_STD를 이용해 input_cnt * output_cnt 형태의 파라미터 생성
    bias = np.zeros([output_cnt])

In [14]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [15]:
pulsar_exec()

Epoch 1 loss=0.155 result=0.086,0.865,0.703,0.776.
Epoch 2 loss=0.137 result=0.086,0.856,0.748,0.799.
Epoch 3 loss=0.132 result=0.086,0.899,0.690,0.781.
Epoch 4 loss=0.129 result=0.086,0.932,0.748,0.830.
Epoch 5 loss=0.121 result=0.086,0.964,0.610,0.747.
Epoch 6 loss=0.125 result=0.086,0.968,0.677,0.797.
Epoch 7 loss=0.129 result=0.086,0.843,0.832,0.838.
Epoch 8 loss=0.125 result=0.086,0.711,0.800,0.753.
Epoch 9 loss=0.126 result=0.086,0.954,0.732,0.828.
Epoch 10 loss=0.129 result=0.086,0.911,0.790,0.846.

Final Test: final accuracy = 0.086,0.911,0.790,0.846


In [16]:
pulsar_exec(adjust_ratio=True)

Epoch 1 loss=0.410 result=0.504,0.967,0.866,0.913.
Epoch 2 loss=0.402 result=0.504,0.922,0.903,0.913.
Epoch 3 loss=0.380 result=0.504,0.975,0.841,0.903.
Epoch 4 loss=0.367 result=0.504,0.965,0.878,0.920.
Epoch 5 loss=0.360 result=0.504,0.967,0.876,0.919.
Epoch 6 loss=0.373 result=0.504,0.926,0.909,0.917.
Epoch 7 loss=0.363 result=0.504,0.724,0.970,0.829.
Epoch 8 loss=0.379 result=0.504,0.931,0.911,0.921.
Epoch 9 loss=0.376 result=0.504,0.956,0.899,0.926.
Epoch 10 loss=0.378 result=0.504,0.968,0.880,0.922.

Final Test: final accuracy = 0.504,0.968,0.880,0.922
